'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS'

In [3]:
import os
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS\\research'

In [4]:
os.chdir("../")
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS'

In [5]:
import os
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
import easyocr
from docClassify.utils.common import read_yaml, create_directories, compute_metrics, scale_bounding_box, create_bounding_box
import torch
from PIL import Image

In [12]:
class Predictor:
    def __init__(self, image_path):
        self.model = LayoutLMv3ForSequenceClassification.from_pretrained("KgModel/IncomeStatement_Cashflow_BalanceStatement__Classifier_LayoutLMv3")
        self.feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
        self.tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
        self.processor = LayoutLMv3Processor(self.feature_extractor, self.tokenizer)
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.reader = easyocr.Reader(['en'])
        self.image_path = image_path

    def predict(self):
    
        ocr_result = self.reader.readtext(str(image_path))
        ocr_page = []
        for bbox, word, confidence in ocr_result:
            ocr_page.append({
                "word": word, "bounding_box": create_bounding_box(bbox)
            })

        #with Image.open(image_path).convert("RGB") as image:

        image = Image.open(image_path).convert("RGB")
        width, height = image.size
        width_scale = 1000 / width
        height_scale = 1000 / height
        words = []
        boxes = []
        for row in ocr_page:
            boxes.append(scale_bounding_box(row["bounding_box"], width_scale, height_scale))
            words.append(row["word"])

        encoding = self.processor(
            image,
            words,
            boxes=boxes,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        
        outputs = self.model(
                    input_ids=encoding["input_ids"].to(self.device),
                    attention_mask=torch.tensor(encoding["attention_mask"]).to(self.device),
                    bbox=torch.tensor(encoding["bbox"]).to(self.device),
                    pixel_values=torch.tensor(encoding["pixel_values"]).to(self.device),
                )
        
        preds_idx = outputs.logits.argmax(axis=1)
        
        return self.model.config.id2label[int(preds_idx[0])]

In [13]:
image_path = r"C:\Users\kbged\Downloads\mlprojects\Document_classifier_with_GithubCICD_FASTAPI_AWS\artifacts\data_ingestion\data\balance sheet\bs1.png"
try:
    predictor = Predictor(image_path)
    prediction = predictor.predict() 
except Exception as e:
    raise e

[2024-02-07 00:17:28,597: WARNING: easyocr: Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.]


C:\Users\kbged\AppData\Local\Temp\ipykernel_22560\2153649251.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(encoding["attention_mask"]).to(self.device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_22560\2153649251.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(encoding["bbox"]).to(self.device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_22560\2153649251.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(encoding["pixel_values"]).to(self.device),


In [14]:
prediction

'balance sheet'

In [6]:

image_path = r"C:\Users\kbged\Downloads\mlprojects\Document_classifier_with_GithubCICD_FASTAPI_AWS\artifacts\data_ingestion\data\balance sheet\bs1.png"


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [11]:
%pwd

'C:\\Users\\kbged\\Downloads\\mlprojects\\Document_classifier_with_GithubCICD_FASTAPI_AWS\\research'

C:\Users\kbged\Miniconda3\envs\doc_classify_aws\Lib\site-packages\transformers\models\layoutlmv3\feature_extraction_layoutlmv3.py:30: FutureWarning: The class LayoutLMv3FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv3ImageProcessor instead.
  warnings.warn(


In [22]:
encoding["input_ids"][0]

tensor([    0, 12341,  2492,  6068,  8157,    12,  1092,    12,  2983,  8835,
           12,  1092,    12,  2983, 20860, 18827, 28092,  1781,  7299, 44321,
         4460,  1781,   290,     6,  1570,   316,     6, 34249,     4,   401,
          158,     6, 39517,     4,   306, 20749,  4748,  4460,  1781,   379,
          195,  6617,     4,   466,   195,  3367,     4,   406,  5143,    12,
         1116,    12,  3698,  1781,   545, 30858,     4,   288, 21458,     4,
          406,  1944,   251,    12,  1279,  5157,  4582,   601,     6,  1244,
         7004,     4,   291,     4,   466,  1944,   251,    12,  1279, 22936,
         1879,  6058,   601,     6,  1366,     6,  1244,  2491,     4,   306,
         2929,     4,   406,  7858, 23474,   629,  1781, 18817,     4,   246,
        11061,     4,   466,  5480,  4460,  1781,   508,     6, 39134,     4,
          246,   365,     6,   398,  1922,     4,   246,  9149,  1781,    96,
         9399,  9023,   753,   195,  4718,     4,   176, 40278, 

C:\Users\kbged\AppData\Local\Temp\ipykernel_16600\2739523392.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask=torch.tensor(encoding["attention_mask"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_16600\2739523392.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  bbox=torch.tensor(encoding["bbox"]).to(device),
C:\Users\kbged\AppData\Local\Temp\ipykernel_16600\2739523392.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pixel_values=torch.tensor(encoding["pixel_values"]).to(device),
C:\Users\kbged\Miniconda3

'balance sheet'

In [34]:
int(preds_idx[0])

0

In [ ]:
from tqdm.notebook import tqdm
import torch
import pandas as pd
import json
from PIL import Image
from transformers import LayoutLMv3FeatureExtractor, LayoutLMv3TokenizerFast, LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader


class Predict:
    def __init__(self, config: DataPredictionConfig):
        self.feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
        self.tokenizer = LayoutLMv3TokenizerFast.from_pretrained("microsoft/layoutlmv3-base")
        self.processor = LayoutLMv3Processor(self.feature_extractor, self.tokenizer)
        self.config = config
        self.DOCUMENT_CLASSES = sorted(list(map(lambda p: p.name,Path(self.config.unzip_dir).glob("*"))))    

    def get_train_test_path(self):
        # Convert PosixPath objects to strings
        image_paths=sorted(list(Path(self.config.unzip_dir).glob("*/*.png")))
        image_paths_str = [str(path) for path in image_paths]
        
        # Define labels based on whether the paths contain specific strings
        income_labels = ["income" in path for path in image_paths_str]
        balance_labels = ["balance" in path for path in image_paths_str]
        cashflow_labels = ["cashflow" in path for path in image_paths_str]
        
        # Use any one of the labels as the target for stratified split
        # Here, I'm using income_labels, but you can choose based on your requirements
        train_images_str, test_images_str = train_test_split(image_paths_str, test_size=0.2, stratify=income_labels, random_state=42)
        
        # Convert back to PosixPath objects
        train_images = [Path(path) for path in train_images_str]
        test_images = [Path(path) for path in test_images_str]

        return train_images, test_images
    
    def train(self, train_images, test_images):
        train_dataset = DocumentClassificationDataset(train_images, self.processor)
        valid_dataset = DocumentClassificationDataset(test_images, self.processor)
        
        train_dataloader = DataLoader(
            train_dataset,
            batch_size=1,
            shuffle=True,
            #num_workers=10
        )
        
        valid_dataloader = DataLoader(
            valid_dataset,
            batch_size=1,
            shuffle=False,
            #num_workers=10
        )

        device = "cuda:0" if torch.cuda.is_available() else "cpu"

        n_classes = len(self.DOCUMENT_CLASSES)
        
        model = LayoutLMv3ForSequenceClassification.from_pretrained(
                    "microsoft/layoutlmv3-base",
                    num_labels=n_classes
                )
        model.to(device)

        # load seqeval metric
        #metric = evaluate.load("seqeval")
        model.config.id2label = {k: v for k, v in enumerate(self.DOCUMENT_CLASSES)}
        model.config.label2id = {v: k for k, v in enumerate(self.DOCUMENT_CLASSES)}
        # labels of the model
        ner_labels = list(model.config.id2label.values())
        
        num_epochs = 1
        optimizer = torch.optim.Adam(model.parameters(), lr=0.000001)
        
        # Initialize an empty DataFrame to store the metrics
        columns = ["Epoch", "Training Loss", "Validation Loss", "Precision", "Recall", "F1", "Accuracy"]
        df_metrics = pd.DataFrame(columns=columns)
        
        # Early stopping parameters
        patience = 3 # Number of epochs to wait for improvement
        best_validation_loss = float('inf')
        current_patience = 0
        
        for epoch in range(num_epochs):
            print("Epoch:", epoch)
        
            # Training
            model.train()
            training_loss = 0.0
            num = 0
            for batch in tqdm(train_dataloader):
                labels = torch.Tensor(batch["labels"]).unsqueeze_(0).long().to(device)
                outputs = model(
                    input_ids=batch["input_ids"].to(device),
                    attention_mask=torch.tensor(batch["attention_mask"]).to(device),
                    bbox=torch.tensor(batch["bbox"]).to(device),
                    pixel_values=torch.tensor(batch["pixel_values"]).to(device),
                    labels=batch["labels"].to(device)
                )
                loss = outputs.loss
                training_loss += loss.item()
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                num += 1
        
            print("Training Loss:", training_loss / num)
        
            # Validation
            model.eval()
            preds = []
            labs = []
            validation_loss = 0.0
            num = 0
            for batch in tqdm(valid_dataloader):
                labels = torch.Tensor(batch["labels"]).to(device)
                outputs = model(
                    input_ids=batch["input_ids"].to(device),
                    attention_mask=torch.tensor(batch["attention_mask"]).to(device),
                    bbox=torch.tensor(batch["bbox"]).to(device),
                    pixel_values=torch.tensor(batch["pixel_values"]).to(device),
                    labels=labels
                )
                loss = outputs.loss
                preds_idx = outputs.logits.argmax(axis=1)
                labs.append(labels.tolist())
                preds.append(preds_idx.tolist())
                validation_loss += loss.item()
                num += 1
        
            print("Validation Loss:", validation_loss / num)
            print(preds)
            print(labs)
        
            overall_precision, overall_recall, overall_f1, overall_accuracy = compute_metrics([preds, labs])
            print("Overall Precision:", overall_precision)
            print("Overall Recall:", overall_recall)
        
            # Store metrics in the DataFrame
            metrics_data = {
                "Epoch": epoch,
                "Training Loss": training_loss,
                "Validation Loss": validation_loss,
                "Precision": overall_precision,
                "Recall": overall_recall,
                "F1": overall_f1,
                "Accuracy": overall_accuracy
            }
            #df_metrics = df_metrics.append(metrics_data, ignore_index=True)
            df_metrics.loc[len(df_metrics)] = metrics_data
        
            # Early stopping check
            if validation_loss < best_validation_loss:
                best_validation_loss = validation_loss
                current_patience = 0
            else:
                current_patience += 1
                if current_patience >= patience:
                    print(f"Early stopping! No improvement in validation loss for {patience} consecutive epochs.")
                    break
        
        # Save the DataFrame to a CSV file or do any further analysis
        df_metrics.to_csv("metrics.csv", index=False)
        print(df_metrics)

        return df_metrics
        
        # Convert DataFrame to markdown
        #markdown_table = df_metrics.to_markdown()
        
        # Print the markdown table
        #print(markdown_table)